# Civitai Model Downloader for RunPod 🚀

Easily download one or multiple AI models from [Civitai](https://civitai.com/) directly to your RunPod instance, with automatic filename detection, disk space checks, and robust error handling. 

**Features:**
- Download single or multiple models via Civitai API
- Keeps original filenames and extensions
- Handles network and file errors gracefully
- Logs progress and errors to both file and console
- Checks disk space before downloading
- Configure API key and save directory easily

**How to use:**
1. Set your Civitai API key
2. Define the list of model URLs you want to download
3. Run the download cell and get your models in seconds!

---

_This notebook is ready for production use on RunPod, but works anywhere with Python and Jupyter._

In [ ]:
!pip install requests tqdm

In [ ]:
import requests
import re
import os
import shutil
import logging
import sys
from urllib.parse import urlparse, unquote
from tqdm import tqdm

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler("civitai_download.log"),
        #logging.StreamHandler()
    ]
)

def get_env_var(name, default=None):
    value = os.environ.get(name)
    if value is None:
        return default
    return value

def check_disk_space(path, required_bytes):
    """Check if there is enough free disk space."""
    total, used, free = shutil.disk_usage(path)
    return free >= required_bytes


def download_from_civitai(url, api_key=None, save_dir="."):
    """
    Download a model from Civitai using the API key, keeping the original filename and extension.
    Handles errors and logs all operations.
    Args:
        url: Civitai model download URL
        api_key: Your Civitai API key
        save_dir: Directory where the file will be saved
    Returns:
        The full path of the downloaded file, or None if failed
    """
    api_key = api_key or get_env_var("CIVITAI_API_KEY")
    if not api_key:
        logging.error("API key not provided. Set the CIVITAI_API_KEY environment variable or pass the api_key parameter.")
        return None

    if not os.path.exists(save_dir):
        os.makedirs(save_dir, exist_ok=True)

    if "?token=" not in url:
        download_url = f"{url}?token={api_key}"
    else:
        download_url = url

    try:
        with requests.get(download_url, stream=True, allow_redirects=True, timeout=30) as response:
            response.raise_for_status()
            content_disposition = response.headers.get('Content-Disposition', '')
            filename = None
            if 'filename=' in content_disposition:
                try:
                    if 'filename="' in content_disposition:
                        filename = re.findall('filename="([^"]+)"', content_disposition)[0]
                    else:
                        filename = re.findall('filename=([^;]+)', content_disposition)[0]
                    filename = filename.strip()
                except IndexError:
                    filename = None
            if not filename or '.' not in filename:
                final_url = response.url
                parsed_url = urlparse(final_url)
                path = unquote(parsed_url.path)
                url_filename = os.path.basename(path)
                if url_filename and '.' in url_filename:
                    filename = url_filename
            if not filename or filename == "download" or '.' not in filename:
                model_id = url.split('/')[-1].split('?')[0]
                filename = f"civitai_model_{model_id}.safetensors"
            file_path = os.path.join(save_dir, filename)
            total_size = int(response.headers.get('content-length', 0))
            if total_size > 0 and not check_disk_space(save_dir, total_size):
                logging.error(f"Not enough disk space to download {filename}.")
                return None
            logging.info(f"Downloading {filename} to {file_path}...")
            with requests.get(download_url, stream=True, timeout=60) as download_response:
                download_response.raise_for_status()
                with open(file_path, 'wb') as f:
                    with tqdm(total=total_size, unit='B', unit_scale=True, desc=filename, file=sys.stdout) as pbar:
                        for chunk in download_response.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                                pbar.update(len(chunk))
            logging.info(f"Download completed: {file_path}")
            return file_path
    except requests.RequestException as e:
        logging.error(f"Network error: {e}")
    except OSError as e:
        logging.error(f"File write error: {e}")
    except Exception as e:
        logging.error(f"Unknown error: {e}")
    return None

def download_multiple_models(url_list, api_key=None, save_dir="."):
    """
    Download multiple models from Civitai with error handling and logging.
    Args:
        url_list: List of Civitai model URLs
        api_key: Your Civitai API key
        save_dir: Directory where files will be saved
    Returns:
        List of downloaded file paths
    """
    downloaded_files = []
    for i, url in enumerate(url_list):
        logging.info(f"\nDownloading model {i+1}/{len(url_list)}")
        file_path = download_from_civitai(url, api_key, save_dir)
        if file_path:
            downloaded_files.append(file_path)
        else:
            logging.warning(f"Download failed for {url}")
    return downloaded_files


## 1. Set your Civitai API Key

To download models from Civitai, you need a personal API key. This key authenticates your requests and allows you to access the download endpoints.

**Where to find your API key:**
- Log in to your account on [Civitai](https://civitai.com/)
- Go to your profile settings
- Look for the section called "API Keys" or "Developer/API"
- Copy your API key and paste it in the next cell (replace `your_civitai_api_key_here`)

**Note:** Never share your API key publicly!

In [ ]:
# Set your Civitai API key here (it will be used for all downloads)
import os

CIVITAI_API_KEY = "your_civitai_api_key_here"  # <-- Replace with your real API key
os.environ["CIVITAI_API_KEY"] = CIVITAI_API_KEY

# Optionally, set the directory where models will be saved
SAVE_DIR = "/workspace/models"  # Change this path if you want to save models elsewhere
os.makedirs(SAVE_DIR, exist_ok=True)

print("API key and save directory set.")

## 2. Add your Civitai model URLs and start the download

In the next cell, you can list one or more Civitai model download URLs. You can find these URLs by:
- Navigating to the model page on [Civitai](https://civitai.com/)
- Clicking the download button for the version you want
- Copying the direct download link (it usually looks like `https://civitai.com/api/download/models/123456`)

**How it works:**
- Add as many URLs as you want to the `model_urls` list
- Run the cell to download all models to your chosen directory
- The script will show progress and print the paths of the downloaded files

You can always add or remove URLs from the list as needed.

In [ ]:
# Example: Download one or more models from Civitai

# List of Civitai model download URLs (replace with your own)
model_urls = [
    "https://civitai.com/api/download/models/638187",
    # "https://civitai.com/api/download/models/123456",  # Add more URLs as needed
]

# Launch the download!
downloaded_files = download_multiple_models(model_urls, api_key=os.environ["CIVITAI_API_KEY"], save_dir=SAVE_DIR)

print("\nDownloaded files:")
for f in downloaded_files:
    print(f)